In [40]:
###### Importing important Modules #####

# Data and NBA API
import pandas as pd
import numpy as np

from scipy.stats import norm, gaussian_kde, percentileofscore

pd.options.display.max_columns = None
from nba_api.stats.static import players
from nba_api.stats.endpoints import shotchartdetail
from nba_api.stats.endpoints import playercareerstats

# For Shot Chart
import matplotlib.pyplot as plt
import seaborn as sns

from matplotlib import cm
from matplotlib.patches import Circle, Rectangle, Arc, ConnectionPatch
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection
from matplotlib.colors import LinearSegmentedColormap, ListedColormap, BoundaryNorm
from matplotlib.path import Path
from matplotlib.patches import PathPatch

sns.set_style('white')
sns.set_color_codes()

# for 3NG optimization calcs
from itertools import permutations

In [41]:
##### Function to get 1 players information from NBA_API #####
# This was found on an NBA github project we need to cite

def get_player_shotchartdetail(player_name, season_id):
    """
    Parameters
    ----------
    player_name: name of the player with Capital
    season_id: ex. 2012-13
    """
    
    # player dictionary
    nba_players = players.get_players()
    player_dict = [player for player in nba_players if player['full_name'] == player_name][0]
    
    # career df
    career = playercareerstats.PlayerCareerStats(player_id=player_dict['id'])
    career_df = career.get_data_frames()[0]
    
    # team id during the season
    team_id = career_df[career_df['SEASON_ID'] == season_id]['TEAM_ID']
    
    # shotchardtdetail endpoint
    shotchartlist = shotchartdetail.ShotChartDetail(team_id=int(team_id), 
                                                   player_id=int(player_dict['id']), 
                                                   season_type_all_star='Regular Season', 
                                                   season_nullable=season_id,
                                                   context_measure_simple="FGA").get_data_frames()
    
    return shotchartlist[0], shotchartlist[1]

In [42]:
####### Function to check if point is within region #####

from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
import shapely.affinity

def find_court_location(x_loc, y_loc):
    """
    Takes in player shot location and outputs what region of the
    court they are on. The input must be a Point Object from
    shapely.geometry module.
    """
    
    player_shot_location = Point(x_loc,y_loc)
    
    ### Defining all possible areas of the court
    
    ## Rectangular Areas
    left_corner_three = Polygon([(-250, -47.5), (-250, 92.5), (-220, 92.5), (-220, -47.5)])
    right_corner_three = Polygon([(250, -47.5), (250, 92.5), (220, 92.5), (220, -47.5)])
    
    left_base_jump = Polygon([(-220, -47.5), (-220, 92.5), (-125, 92.5), (-125, -47.5)])
    right_base_jump = Polygon([(220, -47.5), (220, 92.5), (125, 92.5), (125, -47.5)])
    
    left_near_basket = Polygon([(-125, -47.5), (-125, 142.5), (0, 142.5), (0, -47.5)])
    right_near_basket = Polygon([(125, -47.5), (125, 142.5), (0, 142.5), (0, -47.5)])
    
    ## Left & Right Wing 2s
    circle = Point(0,0).buffer(1)
    three_circle = shapely.affinity.scale(circle, 237.5, 237.5)
    
    #making two polygons that contain the area that I want. 
    #If point is inside polygon area and circle then it is in long 2 area
    left_two_polygon = Polygon([(-250, 250), (-250, 92.5), (-125, 92.5), (-125, 250)])
    right_two_polygon = Polygon([(250, 250), (250, 92.5), (125, 92.5), (125, 250)])
     
    ## Left & Right Above FT 2s
    left_two_above_FT = Polygon([(-125, 250), (-125, 142.5), (0, 142.5), (0, 250)])
    right_two_above_FT = Polygon([(125, 250), (125, 142.5), (0, 142.5), (0, 250)])
    
    ## Above 3 point line
    big_circle = shapely.affinity.scale(circle, 287.5, 287.5)
    
    left_three_polygon = Polygon([(-250,300), (-250,0), (-125, 0), (-125, 300)])
    right_three_polygon = Polygon([(250,300), (250,0), (125, 0), (125, 300)])
    
    left_mid_three_polygon = Polygon([(-125,300), (-125,0), (0, 0), (0, 300)])
    right_mid_three_polygon = Polygon([(125,300), (125,0), (0, 0), (0, 300)])
    
    ## Dealing with Restricted Zone
    # Restricted zone below hoop (0,0)
    #left_restricted_line = Rectangle((-40, -47.5), 0, 47.5, linewidth=lw, color="red")
    restrict_below_hoop = Polygon([(-40,-47.5), (-40,0), (40, 0), (40, -47.5)])
    restrict_above_hoop = Polygon([(-40,0), (-40,40), (40, 40), (40, 0)])
    restrict_circle = shapely.affinity.scale(circle, 40, 40)
    
    if left_corner_three.contains(player_shot_location):
        return '3 pointer Left Corner'
    
    elif right_corner_three.contains(player_shot_location):
        return '3 pointer Right Corner'
        
    elif left_base_jump.contains(player_shot_location):
        return '2 pointer Left Baseline'
    
    elif right_base_jump.contains(player_shot_location):
        return '2 pointer Right Baseline'
        
    ##################### Restricted Zone ##################
    elif restrict_below_hoop.contains(player_shot_location):
        return 'Restricted Area'
        
    elif left_near_basket.contains(player_shot_location) & restrict_above_hoop.contains(player_shot_location) & restrict_circle.contains(player_shot_location):
        return 'Restricted Area'
        
    elif right_near_basket.contains(player_shot_location) & restrict_above_hoop.contains(player_shot_location) & restrict_circle.contains(player_shot_location):
        return 'Restricted Area'
        
    elif right_near_basket.contains(player_shot_location):
        return '2 pointer Left near basket'
        
    elif left_near_basket.contains(player_shot_location):
        return '2 pointer Right near basket'
    
    ######################################################
    elif three_circle.contains(player_shot_location) & left_two_polygon.contains(player_shot_location):
        return '2 pointer Left wing'
        
    elif three_circle.contains(player_shot_location) & right_two_polygon.contains(player_shot_location):
        return '2 pointer Right wing'
        
    elif three_circle.contains(player_shot_location) & left_two_above_FT.contains(player_shot_location):
        return '2 pointer Left above FT'
        
    elif three_circle.contains(player_shot_location) & right_two_above_FT.contains(player_shot_location):
        return '2 pointer Right above FT'
        
    elif big_circle.contains(player_shot_location) & left_three_polygon.contains(player_shot_location):
        return '3 pointer Left'
        
    elif big_circle.contains(player_shot_location) & right_three_polygon.contains(player_shot_location):
        return '3 pointer Right'
        
    elif big_circle.contains(player_shot_location) & left_mid_three_polygon.contains(player_shot_location):
        return '3 pointer Middle'
    
    elif big_circle.contains(player_shot_location) & right_mid_three_polygon.contains(player_shot_location):
        return '3 pointer Middle'       
    else:
        return 'Out of bounds'

In [43]:
##### Function to extract 5 players from the NBA API at once ####

def extract_players_API(dict_of_player_and_season):
    '''
    This function takes in a dictionary of players & years and outputs a dictionary of
    player name as the key & their pandas df as the value. This means that we can now
    build out the whole project directly from the API!!!
    '''
    
    dict_of_players_pd_df = {}
    
    for player in dict_of_player_and_season.keys():
        player_shot_chart_df, league_avg = get_player_shotchartdetail(player, dict_of_player_and_season[player])
        #dict_of_players_pd_df[player] = player_shot_chart_df
        
        new_shot_location = []

        for i in range(len(player_shot_chart_df)):
            new_shot_location.append(find_court_location(player_shot_chart_df.LOC_X[i], player_shot_chart_df.LOC_Y[i]))
    
        player_shot_chart_df['new_shot_location'] = new_shot_location
        
        dict_of_players_pd_df[player] = player_shot_chart_df
        
    return dict_of_players_pd_df

In [44]:
#### Function for 3NG of each position on the court for 1 player only ###

def find_all_3NG(player_df):
    '''
    Takes in one players df & outputs their 3NG for each position.
    Function used in all_players_3NG to get 3NG values for all players.
    '''
    
    three_loc = ['3 pointer Left Corner', '3 pointer Middle', '3 pointer Right', 
                 '3 pointer Left',  '3 pointer Right Corner']
    
    shot_make_tracker = {}
    
    for shot_loc in three_loc:
        shot_data = player_df[player_df['new_shot_location'] == shot_loc]
        
        made_counter = 0
        missed_counter = 0
        
        for shot in shot_data['SHOT_MADE_FLAG']:
            if shot == 0:
                missed_counter += 1
            else:
                made_counter += 1
                
        #3NG = 3P * 1.94 - 3Mi * 1.06. 
        three_net_gain = round(made_counter*1.94 - missed_counter*1.06 , 2)
        shot_make_tracker[shot_loc] = [[made_counter, missed_counter], three_net_gain]
        
    return shot_make_tracker

In [45]:
#### Function to apply find_all_3NG for all players ###

def all_players_3NG(all_players_dict):
    '''
    Takes in the dictionary grabbed from the NBA API.
    Outputs a dictionary of players and their 3NG values from each location.
    '''
    
    player_3NG_dict = {}
    
    for player in all_players_dict.keys():
        player_3NG_dict[player] = find_all_3NG(all_players_dict[player])
    
    return player_3NG_dict

In [46]:
#### Function to create a numpy array of the output of all_players_3NG ###

def create_np_array_of_3NG(dict_of_3NG):
    '''
    Takes in a dictionary of 3NG values for players from each 5 positions behind the 3
    point line.
    Outputs a num py array which is necessary for the optimized_player_location function
    to work.
    '''
    index_1 = []
    index_2 = []
    index_3 = []
    index_4 = []
    index_5 = []

    for player in dict_of_3NG.keys():
        for shot_loc in dict_of_3NG[player]:
            if shot_loc == '3 pointer Left Corner':
                index_1.append(dict_of_3NG[player][shot_loc][1])
            if shot_loc == '3 pointer Middle':
                index_2.append(dict_of_3NG[player][shot_loc][1])
            if shot_loc == '3 pointer Right':
                index_3.append(dict_of_3NG[player][shot_loc][1])
            if shot_loc == '3 pointer Left':
                index_4.append(dict_of_3NG[player][shot_loc][1])
            if shot_loc == '3 pointer Right Corner':
                index_5.append(dict_of_3NG[player][shot_loc][1])
                
    return np.array([index_1, index_2, index_3, index_4, index_5])

In [47]:
#### Function to return a dictionary of 5 players and their optimized court location ####

def optimized_player_location(dict_of_3NG):
    '''
    Takes in a dictionary of 3NG values for players from each 5 positions behind the 3
    point line.
    Outputs the player name and where they should stand on the court in
    order to maximize overall 3NG as a unit.
    '''
    
    np_array = create_np_array_of_3NG(dict_of_3NG)
    optimized_shooting_dict = {}
    
    #finds optimized list, index corresponds to player, index value corresponds to row
    best_perm = None
    best_sum = -np.inf
    for perm in permutations(range(5)):
        s = sum([np_array[perm[i],i] for i in range(5)])
        if s > best_sum:
            best_sum = s
            best_perm = perm
    
    # making list of lists of optimized player column # and player shot location row #
    list_of_players_best_shot_locs = []
    for index, value in enumerate(list(best_perm)):
        list_of_players_best_shot_locs.append([index, value])
    
    #converting dict_of_3NG to pandas dataframe
    df = pd.DataFrame(dict_of_3NG)
    
    # appending to optimized_shooting_dict
    for values in list_of_players_best_shot_locs:
        optimized_shooting_dict[df.columns[values[0]]] = df.index[values[1]]
        
    return optimized_shooting_dict 

In [48]:
################# Master Workflow Function ##############

def find_optimized_spacing(five_players_dictionary):
    '''
    Takes in a player dictionary of 5 players as keys & year as values.
    Outputs the player name and where they should stand on the court in
    order to maximize overall 3NG as a unit.
    '''
    
    player_data = extract_players_API(five_players_dictionary)
    player_3NG_data = all_players_3NG(player_data)
    final_spacing = optimized_player_location(player_3NG_data)
    
    return final_spacing


In [50]:
#### Demonstration of final function #####
# Test by inputting any 5 players

great_shooters_dict = {'Joel Embiid': '2020-21', 'Stephen Curry': '2020-21', 'Tyler Herro': '2020-21',
                      'Damian Lillard': '2020-21', 'Jayson Tatum': '2020-21'}

find_optimized_spacing(great_shooters_dict)

{'Joel Embiid': '3 pointer Right Corner',
 'Stephen Curry': '3 pointer Right',
 'Tyler Herro': '3 pointer Left Corner',
 'Damian Lillard': '3 pointer Middle',
 'Jayson Tatum': '3 pointer Left'}

In [51]:
############ The only thing left to do is to convert 3NG values to be per game, which as of now it is not ########